In [ ]:
import json
import base64
import re
import os

def create_json_from_markdown(markdown_path, output_filename="request.json"):
    """
    Parses a markdown file to create a JSON request for a journal entry.

    Args:
        markdown_path (str): The file path to the markdown journal entry.
        output_filename (str): The name of the output JSON file.
    """
    # --- 1. Read and Parse the Markdown File ---
    try:
        with open(markdown_path, 'r') as md_file:
            lines = md_file.readlines()
    except FileNotFoundError:
        print(f"Error: The file '{markdown_path}' was not found.")
        return

    title = ""
    paragraphs = []
    images_to_process = []
    current_paragraph = ""

    # Regular expression to find markdown image tags
    image_regex = re.compile(r'!\[.*\]\((.*?)\)')

    for line in lines:
        line = line.strip()
        
        # Find title
        if line.startswith('### '):
            title = line[4:].strip()
            continue

        # Find image
        image_match = image_regex.match(line)
        if image_match:
            # When an image is found, if there's a current paragraph, add it to the list
            if current_paragraph:
                paragraphs.append(current_paragraph)
                current_paragraph = ""
            
            image_path = image_match.group(1)
            # The position is after the last paragraph found (0-indexed)
            position = len(paragraphs) - 1
            images_to_process.append({"path": image_path, "position": position})
            continue

        # Collate paragraphs
        if line:
            current_paragraph += line + " "
        elif current_paragraph:
            # A blank line signifies the end of a paragraph
            paragraphs.append(current_paragraph.strip())
            current_paragraph = ""
    
    # Add any remaining paragraph
    if current_paragraph:
        paragraphs.append(current_paragraph.strip())

    # The full text is all paragraphs joined together
    full_text = "\n\n".join(paragraphs)

    # --- 2. Process and Encode Images ---
    processed_images = []
    for image_info in images_to_process:
        image_path = image_info["path"]
        try:
            image_format = image_path.split('.')[-1]
            with open(image_path, "rb") as image_file:
                base64_bytes = base64.b64encode(image_file.read())
                encoded_image_string = base64_bytes.decode('utf-8')
            
            processed_images.append({
                "encoding": "base64",
                "format": image_format,
                "content": encoded_image_string,
                "position_after_paragraph": image_info["position"]
            })
            print(f"Successfully encoded '{image_path}' to be placed after paragraph {image_info['position']}.")
        except FileNotFoundError:
            print(f"Warning: Image file '{image_path}' not found. Skipping.")
        except Exception as e:
            print(f"An error occurred encoding '{image_path}': {e}")
            
    # --- 3. Structure and Write the JSON File ---
    request_data = {
        "entry_data": {
            "title": title,
            "text": full_text,
        },
        "media_context": {
            "video_emotion": "happy", # Example value
            "video_emotion_confidence": 0.955, # Example value
            "images": processed_images
        }
    }

    try:
        with open(output_filename, 'w') as json_file:
            json.dump(request_data, json_file, indent=2)
        print(f"\nSuccessfully created '{output_filename}'")
    except Exception as e:
        print(f"An error occurred while writing the JSON file: {e}")

In [ ]:
markdown_file_to_parse = "journal.md"
create_json_from_markdown(markdown_file_to_parse)

In [ ]:
import json
import base64
import io
from PIL import Image
import matplotlib.pyplot as plt

def display_image_from_response(file_path: str):
    """
    Reads a JSON response file, decodes a base64 image,
    and displays it using matplotlib.
    """

    # 1. Open and read the JSON file
    with open(file_path, 'r') as f:
        response_data = json.load(f)

    # 2. Extract the base64 string from the 'images' list
    if not response_data.get('images'):
        print("Error: No 'images' key found in the JSON file.")
        return
        
    base64_image_str = response_data['images'][0]

    # 3. Decode the base64 string into bytes
    image_bytes = base64.b64decode(base64_image_str)

    # 4. Create an in-memory byte stream and open it as an image
    image_file = io.BytesIO(image_bytes)
    image = Image.open(image_file)

    # 5. Plot the image using matplotlib
    print(f"Displaying image from prompt: \"{response_data.get('prompt', 'N/A')}\"")
    
    plt.figure()
    plt.imshow(image)
    plt.axis('off')  # Hide the axes for a cleaner look
    plt.show()


json_file_path = 'response.json'
display_image_from_response(json_file_path)

In [ ]:
{
  "user_id": "user-uuid",
  "journal_data": {
    "text": "The full, multi-paragraph text of the user's first draft.",
    "user_images": [
      {
        "encoding": "base64",
        "format": "png",
        "content": "base64_string_of_user_image...",
        "position_after_paragraph": 0
      }
    ]
  }
}

In [ ]:
# Output schema
{
  "conversation_id": "string",
  "suggestion": {
    "strategy_used": "string",
    "suggestion_text": "string",
    "highlight_text": "string"
  }
}


### **Project Brief: AI Journaling Chatbot Implementation (Unified Endpoint)**

**To the Coder LLM:**
You are an expert Python developer specializing in FastAPI and AI applications. Your task is to implement the backend logic for a sophisticated chatbot feature for a smart journaling application using a **single, unified endpoint**. The chatbot acts as an "Elaboration Coach" to help enrich writing and as a "Compassionate Listener" for follow-up conversation.

Please use the following detailed specification to generate the Python code. The primary framework is **FastAPI** to handle the request and langchain to handle the interaction with the LLM. LLM used is openAI model. All history (journal drafts, chat messages) is stored on the backend with the help of langchain, associated with a `uuid`.

---

#### **1. High-Level Goal & Chatbot Persona**

* **Goal:** To create an AI assistant that encourages users to write richer, more detailed journal entries and reflect more deeply on their experiences.
* **Persona: "The Compassionate Listener & Elaboration Coach"**.
    * **Tone:** Gentle, curious, validating, and supportive.
    * **Core Directives:** It never gives advice. It asks open-ended questions. It uses the journal's context to be relevant. It preserves the user's original voice.

---

#### **2. Unified API Endpoint Design & Schemas**

A single endpoint will handle all interactions. The backend will determine the user's intent based on the presence of optional fields in the request.

* **Endpoint:** `POST /elaboration-chat`
* **Request Body Schema:**
    ```json
    {
      "uuid": "string",
      "journal_data": {
        "text": "string",
        "user_images": [
          {
            "encoding": "base64",
            "format": "png",
            "content": "string",
            "position_after_paragraph": "integer"
          }
        ]
      },
      "user_chat_input": "string"
    }
    ```
    * **`uuid` (string, required):** The unique session/journal identifier.
    * **`journal_data` (object, optional):** Present for the initial request or when the user has modified the journal text.
    * **`user_chat_input` (string, optional):** Present when the user is sending a conversational message to the chatbot.

* **Response Body Schema:**
    ```json
    {
      "uuid": "string",
      "elaboration_suggestion": {
        "strategy_used": "string",
        "suggestion_text": "string",
        "highlight_text": "string"
      },
      "assistant_response": "string",
      "is_final_message": "boolean"
    }
    ```
    * **`uuid` (string, required):** The session identifier, returned for confirmation.
    * **`elaboration_suggestion` (object, optional):** The coaching prompt. This is the primary response when the user provides `journal_data`. It is `null` otherwise.
    * **`assistant_response` (string, optional):** The chatbot's conversational reply. This is the primary response when the user provides `user_chat_input`. It is `null` otherwise.
    * **`is_final_message` (boolean, required):** Signals to the client if the conversation is considered complete.

---

#### **3. Core AI Logic & LLM Prompts**

##### **Logic for Analyzing Journal Content (The "Elaboration Coach")**

* **System Instruction:**
    > You are an AI Literary Analyst. Your function is to analyze a journal entry and identify the single best paragraph for elaboration. Evaluate each paragraph based on: 1. Visual Richness, 2. Emotional Significance, 3. Narrative Core. You MUST respond with a JSON object containing the `illustratable_paragraphs` as an array of the top 1-4 paragraph indices (based on 40% of total paragraphs).

##### **Logic for Continuous Chat (The "Compassionate Listener")**

* **System Instruction:**
    > You are "Echo," a compassionate and insightful journaling assistant. Your ONLY goal is to ask ONE gentle, open-ended follow-up question or provide a validating concluding remark. **RULES:** DO NOT offer advice. DO NOT share opinions. DO NOT use toxic positivity. Use the provided full session history for context.

---

#### **4. Unified Endpoint Logic & Workflow**

The backend logic for the single `POST /elaboration-chat` endpoint follows this decision tree:

1.  Receive the request containing `uuid` and optional `journal_data` and/or `user_chat_input`.
2.  Load the complete session history for the given `uuid` from the backend's state store (e.g., in-memory dictionary, Redis). The history contains all previous journal text versions and all chat messages.

3.  **Execute one of the following branches:**

    * **A) IF `journal_data` has content (covers user actions 1, 2, and 4):**
        * This is the priority action. The user has written or edited their journal.
        * Update the session history with the new `journal_data.text`.
        * Execute the **"Elaboration Coach"** logic on the new text.
        * Generate an `elaboration_suggestion` (or `null` if the entry is complete).
        * Construct the response with the `elaboration_suggestion` and `assistant_response: null`.

    * **B) ELSE IF `user_chat_input` has content (covers user action 3):**
        * The user is having a conversation.
        * Append the `user_chat_input` to the chat history.
        * Execute the **"Compassionate Listener"** logic using the full session history.
        * Generate an `assistant_response` and determine if it's a final message.
        * Construct the response with the `assistant_response` and `elaboration_suggestion: null`.

4.  Save the updated session history (new text and/or new chat messages) back to the state store associated with the `uuid`.
5.  Return the constructed JSON response to the client.

In [1]:
import os

def extract_python_code_to_markdown(root_dir, output_file):
    """
    Walks through a directory, finds all Python (.py) files,
    extracts their content, and compiles them into a single
    Markdown file.

    Args:
        root_dir (str): The path to the root directory of the project.
        output_file (str): The name of the markdown file to be created.
    """
    # Excluded directories
    excluded_dirs = {'__pycache__', '.venv', 'venv', 'env'}

    try:
        with open(output_file, 'w', encoding='utf-8') as md_file:
            md_file.write("# Project Codebase Summary\n\n")
            print(f"Starting code extraction from '{root_dir}'...")

            # Walk through the directory structure
            for dirpath, dirnames, filenames in os.walk(root_dir):
                # Modify dirnames in-place to skip excluded directories
                dirnames[:] = [d for d in dirnames if d not in excluded_dirs]

                # Sort filenames to maintain a consistent order
                for filename in sorted(filenames):
                    if filename.endswith('.py'):
                        file_path = os.path.join(dirpath, filename)
                        # Create a relative path for cleaner output
                        relative_path = os.path.relpath(file_path, root_dir)

                        print(f"Processing: {relative_path}")

                        # Write a separator and the file path as a header
                        md_file.write(f"---\n\n")
                        md_file.write(f"## File: `{relative_path}`\n\n")
                        md_file.write("```python\n")

                        try:
                            with open(file_path, 'r', encoding='utf-8') as py_file:
                                content = py_file.read()
                                md_file.write(content)
                        except Exception as e:
                            md_file.write(f"# Error reading file: {e}\n")

                        md_file.write("\n```\n\n")

        print(f"\n✅ Successfully generated '{output_file}'!")

    except IOError as e:
        print(f"❌ Error: Could not write to file '{output_file}'. Reason: {e}")
    except Exception as e:
        print(f"❌ An unexpected error occurred: {e}")



start_directory = '../'
output_markdown_file = 'codebase_summary.md'
extract_python_code_to_markdown(start_directory, output_markdown_file)

Starting code extraction from '../'...
Processing: extract.py
Processing: app\__init__.py
Processing: app\config.py
Processing: app\dependencies.py
Processing: app\main.py
Processing: app\schemas.py
Processing: app\logutils\__init__.py
Processing: app\logutils\logger.py
Processing: app\services\__init__.py
Processing: app\services\classification_service.py
Processing: app\services\elaboration_service.py
Processing: app\services\embedding_service.py
Processing: app\services\illustration_service.py
Processing: app\services\model_provider.py
Processing: app\services\session_service.py
Processing: app\services\vlm_service.py

✅ Successfully generated 'codebase_summary.md'!
